In [1]:
include("../src/io_equation.jl")
include("wronskian.jl")

# SIWR Cholera model
models = Dict(
    ["SIWR" => @ODEmodel(
        S'(t) = mu - bi * S(t) * I(t) - bw * S(t) * W(t) - mu * S(t) + a * R(t),
        I'(t) = bw * S(t) * W(t) + bi * S(t) * I(t) - (gam + mu) * I(t),
        W'(t) = xi * (I(t) - W(t)),
        R'(t) = gam * I(t) - (mu + a) * R(t),
        y(t) = k * I(t)
    ),
    "Pharm" => @ODEmodel(
        x0'(t) = a1 * (x1(t) - x0(t)) - (ka * n * x0(t)) / (kc * ka + kc * x2(t) + ka * x0(t)),
        x1'(t) = a2 * (x0(t) - x1(t)),
        x2'(t) = b1 * (x3(t) - x2(t)) - (kc * n * x2(t)) / (kc * ka + kc * x2(t) + ka * x0(t)),
        x3'(t) = b2 * (x2(t) - x3(t)),
        y1(t) = x0(t)
    ),
    "Model 41" => @ODEmodel(
        S'(t) = -b * S(t) * (I(t) + J(t) + q * A(t)),
        E'(t) = b * S(t) * (I(t) + J(t) + q * A(t)) - k * E(t),
        A'(t) = k * (1 - r) * E(t) - g1 * A(t),
        I'(t) = k * r * E(t) - (alpha + g1) * I(t),
        J'(t) = alpha * I(t) - g2 * J(t),
        C'(t) = alpha * I(t),
        y(t) = C(t)
    ),
    "SIWR_multi" => @ODEmodel(
        x_0'(t) = mu - bi * x_0(t) * x_1(t) - bw * x_0(t) * x_2(t) - mu * x_0(t) + a * x_3(t),
        x_1'(t) = bw * x_0(t) * x_2(t) + bi * x_0(t) * x_1(t) - (gam + mu) * x_1(t),
        x_2'(t) = xi * (x_1(t) - x_2(t)),
        x_3'(t) = gam * x_1(t) - (mu + a) * x_3(t),
        y1(t) = k * x_1(t),
        y2(t) = x_0(t) + x_1(t) + x_3(t)
    ),
    "MAPK" => @ODEmodel(
        KS00'(t) = -a00 * K(t) * S00(t) + b00 * KS00(t) + gamma0100 * FS01(t) + gamma1000 * FS10(t) + gamma1100 * FS11(t),
        KS01'(t) = -a01 * K(t) * S01(t) + b01 * KS01(t) + c0001 * KS00(t) - alpha01 * F(t) * S01(t) + beta01 * FS01(t) + gamma1101 * FS11(t),
        KS10'(t) = -a10 * K(t) * S10(t) + b10 * KS10(t) + c0010 * KS00(t) - alpha10 * F(t) * S10(t) + beta10 * FS10(t) + gamma1110 * FS11(t),
        FS01'(t) = -alpha11 * F(t) * S11(t) + beta11 * FS11(t) + c0111 * KS01(t) + c1011 * KS10(t) + c0011 * KS00(t),
        FS10'(t) = a00 * K(t) * S00(t) - (b00 + c0001 + c0010 + c0011) * KS00(t),
        FS11'(t) = a01 * K(t) * S01(t) - (b01 + c0111) * KS01(t),
        K'(t) = a10 * K(t) * S10(t) - (b10 + c1011) * KS10(t),
        F'(t) = alpha01 * F(t) * S01(t) - (beta01 + gamma0100) * FS01(t),
        S00'(t) = alpha10 * F(t) * S10(t) - (beta10 + gamma1000) * FS10(t),
        S01'(t) = alpha11 * F(t) * S11(t) - (beta11 + gamma1101 + gamma1110 + gamma1100) * FS11(t),
        S10'(t) = -a00 * K(t) * S00(t) + (b00 + c0001 + c0010 + c0011) * KS00(t) - a01 * K(t) * S01(t) + (b01 + c0111) * KS01(t) - a10 * K(t) * S10(t) + (b10 + c1011) * KS10(t),
        S11'(t) = -alpha01 * F(t) * S01(t) + (beta01 + gamma0100) * FS01(t) - alpha10 * F(t) * S10(t) + (beta10 + gamma1000) * FS10(t) - alpha11 * F(t) * S11(t) + (beta11 + gamma1101 + gamma1110 + gamma1100) * FS11(t),
        y1(t) = F(t),
        y2(t) = S00(t),
        y3(t) = S01(t),
        y4(t) = S10(t),
        y5(t) = S11(t)
    )]
)

$\require{action}$

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.5.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2020 by The Oscar Development Team
Summary of the model:
State variables: I, R, W, S
Parameter: bi, gam, mu, bw, k, xi, a
Inputs: 
Outputs: y
Summary of the model:
State variables: x0, x2, x3, x1
Parameter: a2, ka, n, b2, kc, b1, a1
Inputs: 
Outputs: y1
Summary of the model:
State variables: I, A, J, S, E, C
Parameter: b, alpha, g2, k, g1, q, r
Inputs: 
Outputs: y
Summary of the model:
State variables: x_1, x_3, x_0, x_2
Parameter: bi, gam, mu, bw, k, xi, a
Inputs: 
Outputs: y1, y2
Summary

Dict{String,ODE{fmpq_mpoly}} with 5 entries:
  "SIWR"       => ODE{fmpq_mpoly}(Multivariate Polynomial Ring in 12 variables …
  "MAPK"       => ODE{fmpq_mpoly}(Multivariate Polynomial Ring in 39 variables …
  "SIWR_multi" => ODE{fmpq_mpoly}(Multivariate Polynomial Ring in 13 variables …
  "Pharm"      => ODE{fmpq_mpoly}(Multivariate Polynomial Ring in 12 variables …
  "Model 41"   => ODE{fmpq_mpoly}(Multivariate Polynomial Ring in 14 variables …

In [2]:
include("wronskian.jl")

for (name, ode) in models
    println("Model $name")
    io_equations = find_ioequations(ode)
    for eq in values(io_equations)
        @time c0 = length(monomial_compress0(eq, ode)[1])
        @time c1 = length(monomial_compress1(eq, ode)[1])
        @time c2 = length(monomial_compress2(eq, ode)[1])
        println("\t $c0  $c1  $c2")
    end
end

Model SIWR
  2.196095 seconds (6.20 M allocations: 661.611 MiB, 22.10% gc time)
  1.850878 seconds (5.79 M allocations: 640.899 MiB, 25.85% gc time)
  2.141047 seconds (9.20 M allocations: 912.381 MiB, 19.12% gc time)
	 771  741  717
Model MAPK
  0.559328 seconds (4.05 M allocations: 319.165 MiB, 17.11% gc time)
  0.190643 seconds (4.05 M allocations: 319.140 MiB)
  0.564173 seconds (4.39 M allocations: 338.813 MiB, 21.36% gc time)
	 4  1  1
  0.739399 seconds (4.38 M allocations: 465.984 MiB, 25.43% gc time)
  0.314647 seconds (4.39 M allocations: 466.635 MiB)
  1.057557 seconds (4.78 M allocations: 490.652 MiB, 45.20% gc time)
	 104  74  62
  1.120485 seconds (5.01 M allocations: 780.804 MiB, 32.37% gc time)
  0.863431 seconds (5.05 M allocations: 782.682 MiB, 19.61% gc time)
  0.849481 seconds (5.47 M allocations: 811.480 MiB, 16.06% gc time)
	 141  116  96
  1.102472 seconds (5.67 M allocations: 1.079 GiB, 16.23% gc time)
  1.089778 seconds (5.73 M allocations: 1.082 GiB, 13.86% gc